In [1]:
import pandas as pd
import numpy as np
import os
pollution = pd.read_csv("all_data.csv", encoding='utf-8')
pollution.info()
pollution.describe()

/var/folders/gd/s1tym5b576ncs0wk41xz2_yc0000gn/T/ipykernel_22811/271499186.py:4: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  pollution = pd.read_csv("all_data.csv", encoding='utf-8')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048576 entries, 0 to 1048575
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   企业编码     1048576 non-null  int64  
 1    排污口编码   1048576 non-null  object 
 2    污染物编码   904156 non-null   object 
 3    记录时间    904156 non-null   object 
 4    污染物浓度   892674 non-null   float64
 5    污染物排放量  821382 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 48.0+ MB


,企业编码,污染物浓度,污染物排放量
count,1.048576e+06,8.926740e+05,8.213820e+05
mean,4.955000e+16,-2.601322e+03,8.344807e+07
std,7.786389e+16,5.595077e+06,7.545968e+10
min,8.942100e+04,-5.157014e+09,-9.268390e+10
25%,1.728540e+13,3.950000e-01,2.500000e-01
50%,1.729084e+13,7.200000e+00,1.331325e+01
75%,1.728621e+17,1.942000e+01,3.864775e+02
max,1.732882e+17,7.659926e+08,6.838900e+13
